## Preprocessing

At this stage, first of all, wrapper functions of multiple different stemming algorithms were written for the data we have.

Before a preprocess operation, separators, operators, punctuations and non-printable characters were removed. Then on the basis of being optional, a normalization can be performed with or without a stop count, or with a different stemming type option.

As a matter of fact, at the last stage, we will compare the result with different parameters in this way.

In [11]:
import re
from trstop import trstop
import string
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
from examples import DATA_PATH, ZEMBEREK_PATH
from pathlib import Path

startJVM(getDefaultJVMPath(), '-ea',
         '-Djava.class.path=%s' % (ZEMBEREK_PATH))

TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
TurkishSentenceNormalizer: JClass = JClass(
    'zemberek.normalization.TurkishSentenceNormalizer'
)


Paths: JClass = JClass('java.nio.file.Paths')


def stem(text: str) -> str:
    morphology = TurkishMorphology.createWithDefaults()

    analysis: java.util.ArrayList = (
        morphology.analyzeAndDisambiguate(text).bestAnalysis()
    )

    pos: List[str] = []
    for i, analysis in enumerate(analysis, start=1):
        pos.append(
            f'{str(analysis.getLemmas()[0])}'
        )
    return ' '.join(pos)


def normalize(text: str) -> str:

    normalizer = TurkishSentenceNormalizer(
        TurkishMorphology.createWithDefaults(),
        Paths.get(str(DATA_PATH.joinpath('normalization'))),
        Paths.get(str(DATA_PATH.joinpath('lm', 'lm.2gram.slm'))),
    )

    return normalizer.normalize(JString(text))


def fps(text: str, n) -> str:
    return ' '.join([w[: n] for w in text.split()])


def preprocess(x, stemming=None):
    x = x.strip()
    x = normalize(x)
    x = remove_punctuation(x)
    x = tokenize(x)
    x = remove_stopwords(x)
    if stemming == 'zemberek':
        x = tokenize(stem(' '.join(x)))
    elif stemming == 'fps5':
        x = tokenize(fps(' '.join(x), 5))
    elif stemming == 'fps7':
        x = tokenize(fps(' '.join(x), 7))

    return x


def remove_punctuation(x):
    return ''.join([w for w in x if w not in string.punctuation])


def tokenize(x):
    return re.split(r'\W+', x)


def remove_stopwords(x):
    return [w for w in x if not trstop.is_stop_word(w)]


## Example for Preprocessing

In [3]:
file_name = 'TTC-3600/TTC-3600_Orj/ekonomi/c (1).txt'

with open(file_name) as file:
    text = file.read()


text

'\ufeff \n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t HGS İhlalli Geçiş Bilgileri Sorgulama Hizmeti ile ihlalli geçişlerin olup olmadığının sorgulanacağının belirtildiği açıklamada, "İhlalli geçişleri var ise son 5 tanesini ve ihlalli geçiş ücretleriyle ihlal ceza tutarlarını görebilmektedir.   Bu hizmetle HGS kullanıcılarının geçiş tarihinden itibaren 15 günlük yasal süresi içinde varsa ihlallerini görerek yeterli bakiyeyi yatırması ve geçişlerin cezaya girmesinin önlenmesi amaçlanmaktadır" ifadesi kullanıldı. \t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t'

In [4]:
preprocess(text)

['',
 'hgs',
 'ihlalli',
 'ihlali',
 'geçişlerin',
 'sorgulanacağının',
 'belirtildiği',
 'ihlali',
 'geçişleri',
 '5',
 'tanesini',
 'ihlali',
 'ücretleriyle',
 'tutarlarını',
 'görebilmektedir',
 'hizmetle',
 'hgs',
 'kullanıcılarının',
 '15',
 'ihlallerini',
 'bakiyeyi',
 'yatırması',
 'geçişlerin',
 'cezaya',
 'girmesinin',
 'önlenmesi',
 'amaçlanmaktadır',
 '']

In [5]:
preprocess(text, stemming='zemberek')

['hgs',
 'ihlal',
 'ihlal',
 'geç',
 'sorgula',
 'belir',
 'ihlal',
 'geç',
 '5',
 'tane',
 'ihlal',
 'ücret',
 'tutar',
 'gör',
 'hizmet',
 'hgs',
 'kullan',
 '15',
 'ihlal',
 'bakiye',
 'yatır',
 'geç',
 'ceza',
 'gir',
 'önle',
 'amaçla']

In [6]:
preprocess(text, stemming='fps5')

['hgs',
 'ihlal',
 'ihlal',
 'geçiş',
 'sorgu',
 'belir',
 'ihlal',
 'geçiş',
 '5',
 'tanes',
 'ihlal',
 'ücret',
 'tutar',
 'göreb',
 'hizme',
 'hgs',
 'kulla',
 '15',
 'ihlal',
 'bakiy',
 'yatır',
 'geçiş',
 'cezay',
 'girme',
 'önlen',
 'amaçl']

In [7]:
preprocess(text, stemming='fps7')

['hgs',
 'ihlalli',
 'ihlali',
 'geçişle',
 'sorgula',
 'belirti',
 'ihlali',
 'geçişle',
 '5',
 'tanesin',
 'ihlali',
 'ücretle',
 'tutarla',
 'görebil',
 'hizmetl',
 'hgs',
 'kullanı',
 '15',
 'ihlalle',
 'bakiyey',
 'yatırma',
 'geçişle',
 'cezaya',
 'girmesi',
 'önlenme',
 'amaçlan']

In [9]:
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import os
import numpy as np


def get_dataset(preprocess):
    dataset = 'TTC-3600/TTC-3600_Orj'


    X, y = [],  []
    target_names = ['ekonomi', 'kultursanat', 'saglik', 'siyaset', 'spor', 'teknoloji']
    for root, directories, files in os.walk(dataset):
        for directory in directories:
            for parent, _, files in os.walk(dataset + '/' + directory):
                y += [directories.index(directory)] * len(files)
                for file in files:
                    with open(parent + '/' + file) as f:
                        X.append(preprocess(f.read()))
    y = np.array(y)
    return X, y

In [12]:
X, y = get_dataset(preprocess=preprocess)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=41)
sns.set()  # use seaborn plotting style

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# Train the model using the training data
model.fit(X_train, y_train)
# Predict the categories of the test data
predicted_categories = model.predict(X_test)


# plot the confusion matrix
mat = confusion_matrix(y_test, predicted_categories)
sns.heatmap(mat.T, square=True, annot=True, fmt="d",
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()


print("The accuracy is {}".format(accuracy_score(y_test, predicted_categories)))

KeyboardInterrupt: 